#Telecom Domain ReadOps Assignment
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

In [0]:
%python
from pyspark.sql.session import SparkSession
print(spark)#already instantiated by databricks
spark1=SparkSession.builder.getOrCreate()
print(spark1)#we instantiated

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

In [0]:
%sql
create catalog if not exists telecom_catalog_assign;
create schema if not exists telecom_catalog_assign.landing_zone;
create volume if not exists telecom_catalog_assign.landing_zone.landing_vol;


In [0]:
%python

for folder in [
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/",
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/",
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1",
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2"
]:
    dbutils.fs.mkdirs(folder)

Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):
a. Volume vs DBFS/FileStore
b. Why production teams prefer Volumes for regulated data

****** NEED TO WORK LATER *******

##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''

In [0]:
%python

customer_csv = """101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
"""

usage_tsv = """customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
"""

tower_logs_region1 = """event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
"""


##2. Filesystem operations
1. Write code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

In [0]:
%python
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", customer_csv, True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv", usage_tsv,True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.csv", tower_logs_region1,True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/tower_logs_region2.csv", tower_logs_region1,True)

dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol")
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage")
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1")
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2")

##3. Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

In [0]:
%python

# 3.1 Read all tower logs using: Path glob filter (example: *.csv) Multiple paths input Recursive lookup
spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/"],header=True,inferSchema=True,sep="|",pathGlobFilter="*.csv",recursiveFileLookup=True).show()

#3.2 Demonstrate these 3 reads separately: Using pathGlobFilter Using list of paths in spark.read.csv([path1, path2]) Using .option("recursiveFileLookup","true")
spark.read.option("header","True").option("inferSchema","True").option("sep","|").option("pathGlobFilter","*.csv").option("recursiveFileLookup","True").format("CSV").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/").show()


******** Compare the outputs and understand when each should be used. ( Need To Work )******** 

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

In [0]:
%python

# Customer
custdf =spark.read.options(header="False",inferSchema="True",sep=",",pathGlobFilter="*.csv",recursiveFileLookup="True").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
custdf.show()

# Usage
usagedf=  spark.read.options(header="True",inferSchema="True",sep="\t",pathGlobFilter="*.csv",recursiveFileLookup="True").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
usagedf.show()


####  Write a note on What changed when we use header or inferSchema with true/false?

<b>header="True" </b> :  It will display the data with user given header <br>
<b>header="False" </b>: It will not display the data with user given header. But defaultly it will show the column name like c0, c1, c2 ....  <br>
<b>inferSchema="True" </b>: It will analyze/Scan the column data and it will show the data type of column.   <br>
<b>inferSchema="False" </b>: It consider and show all the column data type as string (default) <br><br>


#### How schema inference handled “abc” in age?
Age contains both integer and string value so it is considerign the age column as string 


##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

In [0]:
%python
#5.1
custdf.toDF("Id","Name","Age","Location","SimType").show()

#5.2
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,TimestampType
custom_schema = StructType([StructField("cust_id",IntegerType(),False),StructField("voi_mins",IntegerType(),True),StructField("data_mb",IntegerType(),True),StructField("sms_count",IntegerType(),True)])

custom_df1=spark.read.schema(custom_schema).options(header="True", sep="\t").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
custom_df1.show()

custom_df1.printSchema()

#5.3
tower_schema = StructType([
    StructField("eve_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("tower_id", StringType(), True),
    StructField("signal_strength", IntegerType(), True),
    StructField("event_timestamp", TimestampType(), True)
])

tower_df1=spark.read.schema(tower_schema).options(header="True", sep="|" ,pathGlobFilter="*.csv",recursiveFileLookup="True").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")
tower_df1.show()



## 6. More to come (stay motivated)....

##6. Write Operations (Data Conversion/Schema migration) – CSV Format Usecases
1. Write customer data into CSV format using overwrite mode
2. Write usage data into CSV format using append mode
3. Write tower data into CSV format with header enabled and custom separator (|)
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.


In [0]:
%py
#1 : Write customer data into CSV format using overwrite mode
read_cust = spark.read.options(header="False",sep=",").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
write_cust = read_cust.write.options(header='true').mode("overwrite").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/csvout/")
show_cust = spark.read.options(header='true').format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/csvout/")
show_cust.display()

#2 : Write usage data into CSV format using append mode
read_usage = spark.read.options(header="False",sep=",").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv")
append_usage = read_usage.write.csv(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usageout/",header='true',mode='append')
display(append_usage)

#3 : Write tower data into CSV format with header enabled and custom separator (|)
read_tower_df = spark.read.options(header='true',sep='|',inferSchema='true',pathGlobeFilter='.csv',recursiveFileLookup='true').format('csv').load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*")
append_usage = read_tower_df.write.csv(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/towerout/",header='true',mode='append')

#4 : Read the tower data in a dataframe and show only 5 rows.

read_tower_df.show(1)

#5 : Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.
#Yes, we can see the data in notepad++

##7. Write Operations (Data Conversion/Schema migration)– JSON Format Usecases
1. Write customer data into JSON format using overwrite mode
2. Write usage data into JSON format using append mode and snappy compression format
3. Write tower data into JSON format using ignore mode and observe the behavior of this mode
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#1 : Write customer data into JSON format using overwrite mode
read_customer_df = spark.read.options(header="False",sep=",").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv") 
write_customer_json_df = read_customer_df.write.mode("append").json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/jsonout/")

#2 : Write usage data into JSON format using append mode and snappy compression format
read_usage = spark.read.options(header="False",sep=",").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv") 
read_usage.write.mode("append").json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/jsonout/",compression="snappy")

#3 : Write tower data into JSON format using ignore mode and observe the behavior of this mode
read_tower_df = spark.read.options(header='true',sep='|',inferSchema='true',pathGlobeFilter='.csv',recursiveFileLookup='true').format('csv').load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*") 
write_tower_df = read_tower_df.write.mode("ignore").json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/jsonout/")
display(write_tower_df)

#4 : Read the tower data in a dataframe and show only 5 rows.
read_tower_df.show(1)

#5 : Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.
#Yes, we can see the data in notepad++

##8. Write Operations (Data Conversion/Schema migration) – Parquet Format Usecases
1. Write customer data into Parquet format using overwrite mode and in a gzip format
2. Write usage data into Parquet format using error mode
3. Write tower data into Parquet format with gzip compression option
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#1 : Write customer data into Parquet format using overwrite mode and in a gzip format
read_customer_df = spark.read.options(header="False",sep=",").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv") 
write_customer_json_df = read_customer_df.write.mode("overwrite").parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/parquetout/",compression="gzip")

#2:Write usage data into Parquet format using error mode
read_usage = spark.read.options(header="False",sep=",").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv") 
read_usage.write.mode("error").parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/parquetout/")

#3 : Write tower data into Parquet format with gzip compression option
read_tower_df = spark.read.options(header='true',sep='|',inferSchema='true',pathGlobeFilter='.csv',recursiveFileLookup='true').format('csv').load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*") 
write_tower_df = read_tower_df.write.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/parquetout/",compression="gzip")
display(write_tower_df)

#4 : Read the tower data in a dataframe and show only 5 rows.
read_tower_df.show(1)

#5 : Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.
#Yes, we can see the data in notepad++

##9. Write Operations (Data Conversion/Schema migration) – Orc Format Usecases
1. Write customer data into ORC format using overwrite mode
2. Write usage data into ORC format using append mode
3. Write tower data into ORC format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#1: Write customer data into ORC format using overwrite mode
read_customer_df = spark.read.options(header="False",sep=",").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv") 
write_customer_json_df = read_customer_df.write.mode("overwrite").orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/orcout/")

#2 : Write usage data into ORC format using append mode
read_usage = spark.read.options(header="False",sep="\t").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv") 
read_usage.write.mode("append").orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/orcout/")

#3 : Write tower data into ORC format and see the output file structure
read_tower_df = spark.read.options(header='true',sep='|',inferSchema='true',pathGlobeFilter='.csv',recursiveFileLookup='true').format('csv').load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*") 
write_tower_df = read_tower_df.write.mode("append").orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/orcout/")
display(write_tower_df) 

#4 : Read the usage data in a dataframe and show only 5 rows.
read_usage.show(5)

#5 : Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.
#Yes, we can see the data in notepad++

##10. Write Operations (Data Conversion/Schema migration) – Delta Format Usecases
1. Write customer data into Delta format using overwrite mode
2. Write usage data into Delta format using append mode
3. Write tower data into Delta format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.
6. Compare the parquet location and delta location and try to understand what is the differentiating factor, as both are parquet files only.

In [0]:
#1: Write customer data into Delta format using overwrite mode
read_customer_df = spark.read.options(header="False",sep=",").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv") 
write_customer_json_df = read_customer_df.write.mode("overwrite").format("delta").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/deltaout/")

#2: Write usage data into Delta format using append mode
read_usage = spark.read.options(header="True",sep="\t").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv") 
read_usage.write.mode("append").format("delta").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/deltaout/")

#3: Write tower data into Delta format and see the output file structure
read_tower = spark.read.options(header="False",sep=",").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*") 
write_tower = read_tower.write.mode("append").format("delta").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/deltaout/")   

#4:Read the usage data in a dataframe and show only 5 rows.
read_usage.show(5)

#5:Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.
#Yes, we can see the data in notepad++

#6:Compare the parquet location and delta location and try to understand what is the differentiating factor, as both are parquet files only.
#Parquet format : Failed to load 
#Delta format : I can open in the file location and see the data (which is in parquet snappy format)

##11. Write Operations (Lakehouse Usecases) – Delta table Usecases
1.Write customer data using saveAsTable() as a managed table<br>
2.Write usage data using saveAsTable() with overwrite mode<br>
3.Drop the managed table and verify data removal<br>
4.Go and check the table overview and realize it is in delta format in the Catalog.<br>
5.Use spark.read.sql to write some simple queries on the above tables created.<br>

In [0]:

from pyspark.sql.types import StructType, StructField, StringType
usage_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("voice_mins", StringType(), True),
    StructField("data_mb", StringType(), True),
    StructField("sms_count", StringType(), True)
])
#1:Write customer data using saveAsTable() as a managed table
read_customer_df = spark.read.options(header="False",sep=",").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv") 
read_customer_df.write.saveAsTable("telecom_catalog_assign.landing_zone.custtbl",mode='overwrite')

#2:Write usage data using saveAsTable() with overwrite mode
read_usage = spark.read.options(header="True",sep="\t").schema(usage_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv") 
read_usage.write.saveAsTable("telecom_catalog_assign.landing_zone.usagetbl",mode="overwrite")

#3:Drop the managed table and verify data removal
spark.sql("drop table telecom_catalog_assign.landing_zone.usagetbl")

#4.Go and check the table overview and realize it is in delta format in the Catalog.
#yes we can see it is in delta format

#5:Use spark.read.sql to write some simple queries on the above tables created.
spark.sql("select * from telecom_catalog_assign.landing_zone.usagetbl").show()


##12. Write Operations (Lakehouse Usecases) – Delta table Usecases
1.Write customer data using insertInto() in a new table and find the behavior<br>
2.Write usage data using insertTable() with overwrite mode<br>

In [0]:

#1: Write customer data using insertInto() in a new table and find the behavior
#table created for customer
read_customer_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", sep=",").toDF("id","name","age","city","sim_type")
read_customer_df.write.saveAsTable("telecom_catalog_assign.landing_zone.custtbl",mode='overwrite',overwriteSchema=True)
#insert into customer table already created
dlt_tb = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv").toDF("id", "name", "age", "city", "sim_type")
dlt_tb.write.insertInto("telecom_catalog_assign.landing_zone.custtbl")
display(spark.sql("select * from telecom_catalog_assign.landing_zone.custtbl"))

#2:Write usage data using insertTable() with overwrite mode
read_usage = spark.read.options(header="True",sep="\t").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/") 
read_usage.write.insertInto("telecom_catalog_assign.landing_zone.usagetbl",overwrite=True)
spark.sql("select * from telecom_catalog_assign.landing_zone.usagetbl").show()


##13. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data into XML format using rowTag as cust<br>
2. Write usage data into XML format using overwrite mode with the rowTag as usage<br>
3. Download the xml data and open the file in notepad++ and see how the xml file looks like.


In [0]:
#1: Write customer data into XML format using rowTag as cust
xml_cust = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer1.csv").toDF("id", "name", "age", "city", "sim_type")
xml_cust.write.format("xml").mode("overwrite").option("rowTag", "customer").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/xmlout")

#2: Write usage data into XML format using overwrite mode with the rowTag as usage
read_usage = spark.read.options(header="True",sep="\t").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_new.csv") 
read_usage.write.format("xml").mode("overwrite").option("rowTag", "usage").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/xmlout") 

#3: Download the xml data and open the file in notepad++ and see how the xml file looks like.
#Yes, we can see the xml file in notepad++ and it looks like HTML with the row starting and ending with the values provided in rowtag.



##14. Compare all the downloaded files (csv, json, orc, parquet, delta and xml)
1.Capture the size occupied between all of these file formats and list the formats below based on the order of size from small to big.<br>


####Customer.csv format original size 10 MB

%md
<table>
  <thead>
    <tr>
      <th>Format</th>
      <th>File Type</th>
      <th>Memory size</th>
      <th>Human Readable Supported</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>XML</td>
      <td>Semi-structured</td>
      <td>56.2 MB</td>
      <td>Yes</td>
    </tr>
    <tr>
      <td>JSON</td>
      <td>Semi-structured</td>
      <td>20.5 MB</td>
      <td>Yes</td>
    </tr>
    <tr>
      <td>CSV</td>
      <td>Structured</td>
      <td>9.8 MB</td>
      <td>Yes</td>
    </tr>
    <tr>
      <td>ORC</td>
      <td>Structured / Striped</td>
      <td>61.28 KB</td>
      <td>No</td>
    </tr>
    <tr>
      <td>Parquet</td>
      <td>Structured / Nested</td>
      <td>31 KB</td>
      <td>No</td>
    </tr>
    <tr>
      <td>Delta</td>
      <td>Structured / Evolving</td>
      <td>31 KB</td>
      <td>No</td>
    </tr>
  </tbody>
</table>



##15. Try to do permutation and combination of performing Schema Migration & Data Conversion operations like...
1.Read any one of the above orc data in a dataframe and write it to dbfs in a parquet format
2.Read any one of the above parquet data in a dataframe and write it to dbfs in a delta format
3.Read any one of the above delta data in a dataframe and write it to dbfs in a xml format
4.Read any one of the above delta table in a dataframe and write it to dbfs in a json format
5.Read any one of the above delta table in a dataframe and write it to another table

In [0]:
#1: Read any one of the above orc data in a dataframe and write it to dbfs in a parquet format
df1_orc = spark.read.format("orc").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/orcout/part-00000-tid-5274605985532068124-4fb40a1c-a797-4a56-990e-a59d492ce3ca-225-1-c000.snappy.orc") 
display(df1_orc) 
df1_orc.write.mode("overwrite").parquet("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/parquetout/")

#2: Read any one of the above parquet data in a dataframe and write it to dbfs in a delta format
df1_parquet = spark.read.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/parquetout/part-00000-tid-5378167090184321876-f9c2ca52-784d-4f42-a50f-95d657f226d3-236-1.c000.snappy.parquet")
display(df1_parquet) 
df1_parquet.write.format("delta").save("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/deltaout1/")

#3: Read any one of the above delta data in a dataframe and write it to dbfs in a xml format
df1_delta = spark.read.format("delta").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/deltaout1/")
#display(df1_delta) 
df1_delta.write.format("xml").mode("overwrite").option("rowTag", "usage").save("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/xmlout1/")

#4: Read any one of the above delta table in a dataframe and write it to dbfs in a json format
df1_delta = spark.read.format("delta").load("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/deltaout1/")
#display(df1_delta) 
df1_delta.write.format("json").mode("overwrite").save("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/jsonout1/")

#5: Read any one of the above delta table in a dataframe and write it to another table in the same database
df1_delta = spark.read.format("delta").load("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/deltaout1/")
display(df1_delta) 
df1_delta.write.mode("overwrite").saveAsTable("telecom_catalog_assign.landing_zone.usagetbl1")
display(spark.sql("select * from telecom_catalog_assign.landing_zone.usagetbl1"))
     

#####16. Do a final exercise of defining one/two liner of...<br>
1. When to use/benifits csv
- benefits of CSV is simple data transfer/exchange and fast viewing because it is      human-readable
- when we need small data analysis, and supported everywhere
- but it is not suitable for big-data processing because analytics is slow
- best for source data ingestion
- Easy for debugging
- Easy to create
2. When to use/benifits json-
- JSON is semi-structured file type
- API integrations because it supports nested and flexible schemas,
- making it ideal for event and application data.
- it is a dictionary of dictionaries, best for data ingestion.
- widely supported.
3. When to use/benifit orc
- Use ORC for large data,
- intelligent bigdata format,
- read-heavy analytical workloads because it provides excellent compression,
- fast reads, and efficient predicate pushdown.
4. When to use/benifit parquet
- Use Parquet for big-data analytics across multiple platforms because it is a columnar, compressed,
- widely supported format that improves query performance.
5. When to use/benifit delta-
- Use Delta for reliable data lake processing because it adds ACID transactions,
- schema enforcement, time travel.
- scalable incremental loads on top of Parquet.
6. When to use/benifit xml
- Use XML for legacy systems and structured data exchange where strict schemas
- hierarchical data representation are required.
- XML is safer when structure must not break.
7. When to use/benifit delta tables
- Use Delta tables for production-grade lakehouse architectures because they ensure data consistency, versioning, and optimized performance for large datasets.